In [136]:
import pandas as pd
import  category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from category_encoders import *
import  numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

In [154]:
dataset = pd.read_csv("~/Desktop/DataDump/Flights_TRAIN.csv")
dataset.isnull().sum()
dataset = dataset.dropna()

In [155]:

cols_to_encode = ["origin", "dest","op_unique_carrier"]
cols_X = ['dep_dayofweek','op_unique_carrier', 'origin', 'dest',
       'arr_Fog', 'arr_Snow', 'arr_Cold', 'arr_Storm', 'arr_Hail',
          'arr_Precipitation', 'crs_dep_time', 'crs_arr_time',
          'flights_origin', 'flights_dest']
#pd.get_dummies(dataset["op_unique_carrier"])

In [156]:
X = dataset[cols_X]
y = dataset["arr_delay"]
X.head()

,dep_dayofweek,op_unique_carrier,origin,dest,arr_Fog,arr_Snow,arr_Cold,arr_Storm,arr_Hail,arr_Precipitation,crs_dep_time,crs_arr_time,flights_origin,flights_dest
1,0,ZW,MLI,ORD,1.0,3.0,0.0,0.0,0.0,0.0,1045,1150,7,268
2,0,OO,ORD,LGA,1.0,3.0,0.0,0.0,0.0,0.0,1930,2243,302,58
3,0,OO,ORD,LGA,1.0,3.0,0.0,0.0,0.0,0.0,1130,1438,251,132
4,0,OO,GSP,LGA,1.0,3.0,0.0,0.0,0.0,0.0,600,759,11,131
5,0,OO,ORD,LGA,1.0,3.0,0.0,0.0,0.0,0.0,1230,1540,286,136


In [157]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1234)

In [158]:
X_train = X_train.reset_index(drop=True)     
y_train = y_train.reset_index(drop=True)

In [159]:
encoder = TargetEncoder(cols=cols_to_encode)

In [160]:
X_train_fitted = X_train.iloc[0:0,:]

In [161]:
X_train_fitted

,dep_dayofweek,op_unique_carrier,origin,dest,arr_Fog,arr_Snow,arr_Cold,arr_Storm,arr_Hail,arr_Precipitation,crs_dep_time,crs_arr_time,flights_origin,flights_dest


In [162]:
kf = KFold(n_splits = 5)
for tr_ind, val_ind in kf.split(X_train, y_train):
    encoder.fit(X_train.loc[tr_ind], y_train.loc[tr_ind])
    X_train_fitted = X_train_fitted.append(encoder.transform(X_train.loc[val_ind]))

In [163]:
encoder.fit(X_train, y_train)
X_test_fitted = encoder.transform(X_test)

In [169]:
max_depth = [10, 15, 20]
n_estimators = [10, 25, 50]
params = {"max_depth": max_depth, "n_estimators": n_estimators}

In [170]:
param_comb = 10

In [171]:
mod = RandomForestClassifier()

In [ ]:
#preserving same cv
random_search = RandomizedSearchCV(mod, param_distributions=params, n_iter=param_comb, n_jobs=2, verbose=3, random_state=0, cv = kf.split(X_train, y_train))
random_search.fit(X_train_fitted, y_train)
print(random_search.best_score_)

/home/joaolrossi/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 9 candidates, totalling 45 fits
